In [1]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup, NavigableString

In [2]:
## We read the excel file from the 'TextExtraction_Git' notebook
meds = pd.read_excel('Medicina.xlsx')

In [3]:
meds

,id,idno,date,titulo,compr_score,campo,autor,autor_key
0,1025107,http://gallica.bnf.fr/ark:/12148/bpt6k1025107h,1862,Observation sur l'action physiologique du café...,0.64,610 - Sciences médicales. Médecine,Offret,10480124
1,1025126,http://gallica.bnf.fr/ark:/12148/bpt6k1025126v,1779,Recherches sur la cause des affections hypocon...,0.66,610 - Sciences médicales. Médecine,"Revillon, Claude",10413078
2,1025127,http://gallica.bnf.fr/ark:/12148/bpt6k10251278,1874,De la valeur des symptômes en pathologie mentale,0.67,610 - Sciences médicales. Médecine,"Calmette, Émile (1851-1934)",10495335
3,1025130,http://gallica.bnf.fr/ark:/12148/bpt6k1025130r,1874,Études sur le goître et le crétinisme,0.67,610 - Sciences médicales. Médecine,"Parchappe, Maximien (1800-1866)",12139929
4,1025133,http://gallica.bnf.fr/ark:/12148/bpt6k10251330,1859,Traité clinique et thérapeutique de l'hystérie,0.67,610 - Sciences médicales. Médecine,"Briquet, Pierre (1796-1881)",11894062
...,...,...,...,...,...,...,...,...
179,5407471,http://gallica.bnf.fr/ark:/12148/bpt6k5407471k,1889,Notes et observations cliniques,0.66,610 - Sciences médicales. Médecine,"Lancial, Louis",10475159
180,5407472,http://gallica.bnf.fr/ark:/12148/bpt6k54074720,1848,Rapport adressé à la Commission administrative...,0.66,610 - Sciences médicales. Médecine,"Marchant, Léon",12054948
181,5407473,http://gallica.bnf.fr/ark:/12148/bpt6k5407473d,1878,"Nécrologie médicale raisonnée, ou Recherches s...",0.67,610 - Sciences médicales. Médecine,"Marmisse, Géraud (1824-18..?)",12991814
182,5407477,http://gallica.bnf.fr/ark:/12148/bpt6k54074772,1883,Conférence faite au dispensaire municipal de l...,0.67,610 - Sciences médicales. Médecine,"Fredet, Gilbert Edmond (1840-1912)",10524928


In [4]:
## Function that extracts and save text between to tags in a xml file
def between(cur, end):
    while cur and cur != end:
        if isinstance(cur, NavigableString):
            text = cur.strip().replace('\n', '').replace('\xa0', '') ## Replacing line braks and some decodification errors
            if len(text) > 0:  ## checks if there is text between the tags and yields it
                yield text
        cur = cur.next_element

In [5]:
# Function that retrieves text from between function
def recup_textos(cur, end):
    pag = ' '.join(text for text in between(cur.next_sibling, end))
    return pag

In [6]:
# Test for 1 document para 1 documento
## To exract the text for each page
### First we open the document and make the soup
file = meds.id[0]
a = open('./TGB_Sample/' + str(file) + '_r.xml', 'r', encoding="utf8")
soup = BeautifulSoup(a, 'lxml')
### Retrieve the page tags (this could be changed to any tag)
pags = [tag for tag in soup.find_all('pb')]
### Getting the last tag
ultima_tag = [i for i in soup.find_all(True)][-1]
### adding the last tag for the last page
pags.append(ultima_tag)
### We run from the tags iteratiely appending also the metadata
new_data = []
for i in range(len(pags)-1):
    texto = recup_textos(pags[i], pags[i+1])
    pagina = pags[i]['xml:id']
    if len(texto) > 0:
        new_data.append([file,pagina,texto])

In [7]:
## Checking if the text segmentation was successful
new_data

[[1025107,
  'PAG_00000007',
  'Observations SUR L’action psysiologique du café, SELON SES DIVERSES TORRÉFACTIONS,PAR OFFRET,PHARMACIEN A NANTES NANTES, IMPRIMERIE DE VINCENT FOREST ET ÉMILE GRIMAUD, PLACE DU COMMERCE, 1. 1862.'],
 [1025107,
  'PAG_00000009',
  '1 Observations SUR L’action physiologique du café. Ayant été frappé, en 1849, ainsi que quelques médecins, del’effet que produisit sur moi l’emploi du café, selon ses diversestorréfactions , j’en ai renouvelé plusieurs fois l’observation depuiscette époque, tant sur moi que sur mes élèves qui ont bien voulus’y soumettre: et les faits observés en 1849, ont été les mêmes. Partant de ce point, et ayant à coeur d’aider la science, commepeut-être de découvrir un agent thérapeutique capable d’apporterun calme à nos souffrances, j’ai fait part de mes observations à desmédecins, désirant ardemment que des personnes compétentesvinssent les approuver ou les réfuter. Quelque prévenues qu’elles fussent contre, elles ont été forcéesde se re

In [ ]:
# For all the documents
new_data = []
for i in range(len(meds)):
    file = meds.id[i]
    a = open('./TGB_Sample/' + str(file) + '_r.xml', 'r', encoding="utf8")
    soup = BeautifulSoup(a, 'lxml')
    ### Getting all the tag pages
    pags = [tag for tag in soup.find_all('pb')]
    ### Getting the last tag
    ultima_tag = [i for i in soup.find_all(True)][-1]
    ### adding the last tag to the set in order to retrieve the last page
    pags.append(ultima_tag)
    ### We run from the tags iteratiely appending also the metadata
    for k in range(len(pags)-1):
        texto = recup_textos(pags[k], pags[k+1])
        pagina = pags[k]['xml:id']
        if len(texto) > 0:
            new_data.append([meds.id[i], meds.idno[i], meds.date[i], meds.titulo[i], meds.compr_score[i], meds.campo[i], 
                             meds.autor[i], meds.autor_key[i], pagina, texto])

In [28]:
## Saving the text segmentation
meds_pags = pd.DataFrame(new_data, columns = ['id', 'idno', 'date', 'titulo', 'compr_score',
                                              'campo', 'autor', 'autor_key', 'pagina', 'texto'])

In [29]:
meds_pags

,id,idno,date,titulo,compr_score,campo,autor,autor_key,pagina,texto
0,1025107,http://gallica.bnf.fr/ark:/12148/bpt6k1025107h,1862,Observation sur l'action physiologique du café...,0.64,610 - Sciences médicales. Médecine,Offret,10480124,PAG_00000007,Observations SUR L’action psysiologique du caf...
1,1025107,http://gallica.bnf.fr/ark:/12148/bpt6k1025107h,1862,Observation sur l'action physiologique du café...,0.64,610 - Sciences médicales. Médecine,Offret,10480124,PAG_00000009,1 Observations SUR L’action physiologique du c...
2,1025107,http://gallica.bnf.fr/ark:/12148/bpt6k1025107h,1862,Observation sur l'action physiologique du café...,0.64,610 - Sciences médicales. Médecine,Offret,10480124,PAG_00000010,— 4 — distillation que je concentrai en consis...
3,1025107,http://gallica.bnf.fr/ark:/12148/bpt6k1025107h,1862,Observation sur l'action physiologique du café...,0.64,610 - Sciences médicales. Médecine,Offret,10480124,PAG_00000011,"— 5 — En octobre 1855, forcé par les circonsta..."
4,1025107,http://gallica.bnf.fr/ark:/12148/bpt6k1025107h,1862,Observation sur l'action physiologique du café...,0.64,610 - Sciences médicales. Médecine,Offret,10480124,PAG_00000012,"— 6 — Je crois que le café noir, dit des amate..."
...,...,...,...,...,...,...,...,...,...,...
22467,5407549,http://gallica.bnf.fr/ark:/12148/bpt6k54075493,1862,"De la statistique mortuaire, par les médecins ...",0.67,610 - Sciences médicales. Médecine,"Marmisse, Géraud (1824-18..?)",12991814,PAG_00000004,mortels survenus sur la voie publique ou dans ...
22468,5407549,http://gallica.bnf.fr/ark:/12148/bpt6k54075493,1862,"De la statistique mortuaire, par les médecins ...",0.67,610 - Sciences médicales. Médecine,"Marmisse, Géraud (1824-18..?)",12991814,PAG_00000005,scène est éclairée souvent pour tout le monde;...
22469,5407549,http://gallica.bnf.fr/ark:/12148/bpt6k54075493,1862,"De la statistique mortuaire, par les médecins ...",0.67,610 - Sciences médicales. Médecine,"Marmisse, Géraud (1824-18..?)",12991814,PAG_00000006,ou une péricardite; le même traitement sur la ...
22470,5407549,http://gallica.bnf.fr/ark:/12148/bpt6k54075493,1862,"De la statistique mortuaire, par les médecins ...",0.67,610 - Sciences médicales. Médecine,"Marmisse, Géraud (1824-18..?)",12991814,PAG_00000007,"sie, la syncope, l’hémorrhagie, l’épuisement o..."


In [50]:
### Minimum number of pages for a document
meds_pags.id.value_counts(ascending=False).min()

6

In [32]:
# Saving to an excel file
meds_pags.to_excel('Medicos_segmentados.xlsx', index = False)

In [10]:
meds_pags = pd.read_excel('Medicos_segmentados.xlsx')

In [28]:
## Text length 
tamaño_texto = []
for i in range(len(meds_pags)):
    try:
        tamaño_texto.append(len(meds_pags.texto[i].lower().replace(',', ', ').split()))
    except:
        tamaño_texto.append(0)

In [30]:
meds_pags['Tamaño_Texto'] = tamaño_texto

In [32]:
## mean text length
meds_pags.Tamaño_Texto.mean()

243.5752936988252